In [1]:
from flask import Flask, request
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
import mongodb as md
from flask_jsonpify import jsonpify

In [2]:
movies = md.read_mongo("finalyearproject","movies",True)

In [12]:
def getSingleGenre(genre,genre_movies):
    percentile=0.85
    df = genre_movies[genre_movies['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    print(m)
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['tmdb','title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')

    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False)
    qualified = qualified.drop_duplicates(['tmdb']).dropna()
    return qualified



genres = ["Action","Drama"]
movies['year'] = pd.to_datetime(movies['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

s = movies.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
genre_movies = movies.drop('genres', axis=1).join(s)
allGenres = []
# for genre in genres:
#     allGenres.append()



/var/folders/mr/3szmdm1d3sj80jkk3tl1c_t40000gp/T/ipykernel_37430/3856370136.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movies.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)


In [13]:
getSingleGenre("Action",genre_movies)

0.0


/var/folders/mr/3szmdm1d3sj80jkk3tl1c_t40000gp/T/ipykernel_37430/3856370136.py:13: RuntimeWarning: invalid value encountered in double_scalars
  qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)


,tmdb,title,year,vote_count,vote_average,popularity,wr
174,111,Scarface,1983,1,5,52.966,5.0
1389,60304,Hansel & Gretel: Witch Hunters,2013,1,5,89.806,5.0
1396,102651,Maleficent,2014,1,5,97.558,5.0
1458,172385,Rio 2,2014,1,5,79.664,5.0
16,155,The Dark Knight,2008,1,4,119.599,4.0
139,638507,How to Train Your Dragon: Homecoming,2019,1,4,64.517,4.0
253,280,Terminator 2: Judgment Day,1991,1,4,61.713,4.0
113,670,Oldboy,2003,1,3,35.230,3.0
133,16869,Inglourious Basterds,2009,1,3,100.285,3.0
136,634649,Spider-Man: No Way Home,2021,1,3,6064.019,3.0
